<a href="https://colab.research.google.com/github/simon-clematide/casdmit-fs21/blob/master/zora_dewey_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dewey-Klassifikation mit fasttext
Dieses Notebook demonstriert, wie einfach man ein gutes Klassifikations-Modell mit fastText trainieren kann.
Wir arbeiten mit der fasttext Python-Bibliothek.
Aus Effizienzgründen arbeiten wir hier mit einem kleineren Trainingsdatensatz.

## Das Python fasttext Package installieren
Aktuellere Version hat [Bug](https://stackoverflow.com/questions/61787119/fasttext-0-9-2-why-is-recall-nan) in der Evaluationsfunktion korrigiert 

In [1]:
# ! pip install fasttext # schnell zu installieren, aber hat Bug bei test_label()
! pip install git+https://github.com/facebookresearch/fastText.git  # braucht mehr Zeit fürs Kompilieren

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-bqx440ss
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-bqx440ss
  Resolved https://github.com/facebookresearch/fastText.git to commit 0622aad8571861d290b237e83e04e9a07a28839d
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4384420 sha256=df108c37d26d4d8502bac4ee51762e35df8b309960106c38302d832acbaa2f55
  Stored in directory: /tmp/pip-ephem-wheel-cache-dclezgzh/wheels/3d/b7/8f/678f8b04408b579e2acad5f2416797aee4ebfe3529049a03b2
Successfully built fasttext


In [ ]:
! pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! python3 -m spacy download en_core_web_sm

2023-05-10 19:07:53.543241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 19:07:54.895004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import logging
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes("parser", "ner")

['parser', 'ner']

# Datenset: Zufällig ausgewählte

In [ ]:
! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/zora-eng-dewey.fasttext.tsv -o zora-eng-dewey.fasttext.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.1M  100 15.1M    0     0  8286k      0  0:00:01  0:00:01 --:--:-- 8282k


### Format des Datensets 
 - Pro tabulator-separierte Zeile gibt es 2 Spalten
 - Spalte 1: Deweylabels
 - Spalte 2: Titel und Abstract untokenisiert

In [ ]:
! head -n 10 zora-eng-dewey.fasttext.tsv

__label__300	Orangutan activity budgets and diet: a comparison between species, populations and habitats The chapter examines differences in the activity budgets of wild orangutans (Pongo spp.) within and between a large number of study sites in Sumatra and Borneo. The authors of the chapter found that each orangutan population appeared to follow one of two distinct foraging strategies: either (1) ‘sit and wait’, in which orangutans aim to minimize their energy expenditure by spending long periods of time resting and relatively short periods feeding and travelling; or (2) ‘search and find’ in which orangutans aim to maximize their energy intake by resting little and mainly feeding or moving in search of food. Orangutans adopt the first strategy in mixed-dipterocarp forests characterized by mast-fruiting events and irregular fruit availability; and adopt the second strategy in swamp forests with a regular supply of fruit, or in dryland forests with high strangling-fig density. The chapt

### Statistiken zum Datenset

In [ ]:
! wc zora-eng-dewey.fasttext.tsv

   10267  2270564 15910028 zora-eng-dewey.fasttext.tsv


In [ ]:
!  cut -f 1 < zora-eng-dewey.fasttext.tsv | sort | uniq -c | sort -rn 

   4020 __label__610
    927 __label__570 __label__610
    603 __label__570
    537 __label__330
    447 __label__150
    440 __label__530
    355 __label__910
    328 __label__570 __label__590
    288 __label__570 __label__630
    260 __label__540
    226 __label__000
    193 __label__510
    180 __label__580
    130 __label__170 __label__610
    112 __label__300
    104 __label__320
     83 __label__142 __label__610
     77 __label__560
     65 __label__370
     65 __label__070
     57 __label__000 __label__410
     50 __label__142
     47 __label__820
     46 __label__142 __label__570 __label__610
     41 __label__340 __label__610
     40 __label__570 __label__610 __label__600
     40 __label__100
     37 __label__610 __label__540
     37 __label__340
     28 __label__800 __label__470 __label__410 __label__440 __label__460 __label__450
     26 __label__490 __label__890 __label__410
     23 __label__900
     19 __label__170 __label__330
     17 __label__340 __label__610 __label__510


In [ ]:
def lemmatize_tsv(inputfile, outputfile, spacy_nlp, limit=999999):
    with open(outputfile,"w",encoding="utf-8")as output:
        with open(inputfile,"r",encoding="utf-8") as input:
            for i,line in enumerate(input):
                labels, text = line.strip().split("\t")
                doc = nlp(text)
                print(labels,' '.join(token.lemma_ for token in doc).lower(),sep="\t",file=output)
                if i > limit:
                    break
                if i % 100 == 0:
                    print(f"Processed {i} records")


In [ ]:
lemmatize_tsv("zora-eng-dewey.fasttext.tsv","zora-eng-dewey-10.lemmatized.fasttext.tsv",nlp,limit=10)

Processed 0 records


In [ ]:
! head zora-eng-dewey-10.lemmatized.fasttext.tsv

__label__300	orangutan activity budget and diet : a comparison between specie , population and habitat the chapter examine difference in the activity budget of wild orangutan ( pongo spp . ) within and between a large number of study site in sumatra and borneo . the author of the chapter find that each orangutan population appear to follow one of two distinct forage strategy : either ( 1 ) ' sit and wait ' , in which orangutan aim to minimize their energy expenditure by spend long period of time rest and relatively short period feed and travelling ; or ( 2 ) ' search and find ' in which orangutan aim to maximize their energy intake by rest little and mainly feed or move in search of food . orangutan adopt the first strategy in mixed - dipterocarp forest characterize by mast - fruit event and irregular fruit availability ; and adopt the second strategy in swamp forest with a regular supply of fruit , or in dryland forest with high strangle - fig density . the chapter propose that the de

In [5]:
# Download precomputed lemmatized data
! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/zora-eng-dewey.lemmatized.fasttext.tsv -o zora-eng-dewey.lemmatized.fasttext.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.0M  100 15.0M    0     0  3706k      0  0:00:04  0:00:04 --:--:-- 3706k


In [ ]:
lemmatize_tsv("zora-eng-dewey.fasttext.tsv","zora-eng-dewey.lemmatized.fasttext.tsv",nlp)

Processed 0 records
Processed 100 records
Processed 200 records
Processed 300 records
Processed 400 records
Processed 500 records
Processed 600 records
Processed 700 records
Processed 800 records
Processed 900 records
Processed 1000 records
Processed 1100 records
Processed 1200 records
Processed 1300 records
Processed 1400 records
Processed 1500 records
Processed 1600 records
Processed 1700 records
Processed 1800 records
Processed 1900 records
Processed 2000 records
Processed 2100 records
Processed 2200 records
Processed 2300 records
Processed 2400 records
Processed 2500 records
Processed 2600 records
Processed 2700 records
Processed 2800 records
Processed 2900 records
Processed 3000 records
Processed 3100 records
Processed 3200 records
Processed 3300 records
Processed 3400 records
Processed 3500 records
Processed 3600 records
Processed 3700 records
Processed 3800 records
Processed 3900 records
Processed 4000 records
Processed 4100 records
Processed 4200 records
Processed 4300 records


In [6]:
! head zora-eng-dewey.lemmatized.fasttext.tsv

__label__300	orangutan activity budget and diet : a comparison between specie , population and habitat the chapter examine difference in the activity budget of wild orangutan ( pongo spp . ) within and between a large number of study site in sumatra and borneo . the author of the chapter find that each orangutan population appear to follow one of two distinct forage strategy : either ( 1 ) ' sit and wait ' , in which orangutan aim to minimize their energy expenditure by spend long period of time rest and relatively short period feed and travelling ; or ( 2 ) ' search and find ' in which orangutan aim to maximize their energy intake by rest little and mainly feed or move in search of food . orangutan adopt the first strategy in mixed - dipterocarp forest characterize by mast - fruit event and irregular fruit availability ; and adopt the second strategy in swamp forest with a regular supply of fruit , or in dryland forest with high strangle - fig density . the chapter propose that the de

## Aufteilen der Daten in Trainings- und Testdaten
Erstellen von Training und Testdaten (Originaldaten sind zufällig geordnet)

In [7]:
! head -n 9000 < zora-eng-dewey.lemmatized.fasttext.tsv > zora-eng-dewey.lemmatized.fasttext.train.tsv
! tail -n 1000 < zora-eng-dewey.lemmatized.fasttext.tsv > zora-eng-dewey.lemmatized.fasttext.test.tsv

In [8]:
! echo TRAINING DATA STATISTICS
! cut -f 1 < zora-eng-dewey.lemmatized.fasttext.train.tsv | sort | uniq -c | sort -rn |head
! echo TEST DATA STATISTICS
! cut -f 1 < zora-eng-dewey.lemmatized.fasttext.test.tsv | sort | uniq -c | sort -rn |head

TRAINING DATA STATISTICS
   3543 __label__610
    806 __label__570 __label__610
    535 __label__570
    475 __label__330
    394 __label__150
    384 __label__530
    306 __label__910
    276 __label__570 __label__590
    258 __label__570 __label__630
    231 __label__540
TEST DATA STATISTICS
    374 __label__610
    104 __label__570 __label__610
     58 __label__570
     50 __label__330
     44 __label__530
     43 __label__910
     40 __label__150
     39 __label__570 __label__590
     25 __label__510
     25 __label__000


# Trainieren von Modell mit Python-Package
 - Dokumentation siehe https://fasttext.cc/docs/en/python-module.html

In [20]:
import fasttext

ImportError: ignored

Word Embeddings auf Wikipedia trainiert auf 50 Dimensionen reduziert

In [15]:
!  curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/wiki.en.50.vec -o wiki.en.50.vec

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184M  100  184M    0     0   9.9M      0  0:00:18  0:00:18 --:--:-- 11.9M


In [23]:

model = fasttext.train_supervised(
    input='zora-eng-dewey.lemmatized.fasttext.train.tsv', 
    pretrainedVectors="wiki.en.50.vec", # vortrainierte word embeddings
    epoch=10,  # Wie oft werden die Trainingsdaten benutzt
    minn=5,    # Minimal Subword-Länge in Buchstaben  
    maxn=5,    # Maximale Subword-Länge in Buchstaben 
    dim=50,    # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords
    lr=1,      # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    ws=10,
    verbose = True
    )

## Inspizieren des gelernten Modells

Welche Labels/Klassen kennt das Modell?

In [24]:
print(model.labels)

['__label__610', '__label__570', '__label__330', '__label__150', '__label__530', '__label__910', '__label__590', '__label__540', '__label__630', '__label__000', '__label__142', '__label__510', '__label__580', '__label__170', '__label__300', '__label__410', '__label__320', '__label__340', '__label__560', '__label__070', '__label__370', '__label__820', '__label__600', '__label__100', '__label__900', '__label__490', '__label__470', '__label__800', '__label__460', '__label__440', '__label__450', '__label__890', '__label__390', '__label__180', '__label__700', '__label__950', '__label__290', '__label__790', '__label__200', '__label__480', '__label__430', '__label__230', '__label__930', '__label__360', '__label__400', '__label__830', '__label__921', '__label__860', '__label__850', '__label__810', '__label__840']


Einen String klassifizieren und die Wahrscheinlichkeitsverteilung über allen möglichen Dewey erhalten:

In [25]:
result = model.predict("interpersonal problems associate with multidimensional personality questionnaire traits in woman ",  
              k=5  # Gib die 5 besten Klassen aus
              )
for label,prob in zip(*result):
    print(label, round(prob,3))

__label__150 1.0
__label__330 0.0
__label__370 0.0
__label__610 0.0
__label__170 0.0


Systematisches Testen des Minimodells auf Testdaten

In [28]:
model.test("zora-eng-dewey.lemmatized.fasttext.test.tsv",k=3,threshold=0.25)

(1000, 0.7992957746478874, 0.712156862745098)

In [29]:
def print_results(N, p, r):
    "Pretty print performance: N=Number of Samples, P/R@1=Precision/Recall of best prediction Acc=Accuracy "
    print(f"N\t{N}")
    print(f"P@k\t{p:.2f}")
    print(f"R@k\t{r:.2f}")
    print(f"Acc\t{r:.2f}")

In [30]:
print_results(*model.test("zora-eng-dewey.lemmatized.fasttext.test.tsv",k=3,threshold=0.25))

N	1000
P@k	0.80
R@k	0.71
Acc	0.71


Detaillierte Evaluation zu jedem einzelnen Label:
 - Precision: Anteil korrekter Klassifikationen einer Klasse
 - Recall: Anteil korrekt klassifizierter Elemente einer Klasse
 - f1score: Harmonisches Mittel von Precision und Recall

In [36]:
data = model.test_label('zora-eng-dewey.lemmatized.fasttext.test.tsv',k=3, threshold=0.35)
sorted_data = sorted(data.items(), key=lambda x: x[1]['f1score'], reverse=True)
print(sorted_data)
for label, perf in sorted_data:
    print(label, perf)

[('__label__810', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__850', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__860', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__921', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__400', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__360', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__430', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__530', {'precision': 0.9555555555555556, 'recall': 0.9347826086956522, 'f1score': 0.945054945054945}), ('__label__070', {'precision': 1.0, 'recall': 0.8571428571428571, 'f1score': 0.9230769230769231}), ('__label__000', {'precision': 0.8333333333333334, 'recall': 0.6896551724137931, 'f1score': 0.7547169811320755}), ('__label__830', {'precision': nan, 'recall': nan, 'f1score': nan}), ('__label__510', {'precision': 0.8571428571428571, 'recall': 0.96, 'f1score': 0.9056603773584906}), ('__label__840', {'pr

## Vorhersagen und Wahrheit anzeigen

In [ ]:
!ls -lh

total 4.2G
-rw-r--r-- 1 root root  83M May 10 22:34 model_subset.txt
drwxr-xr-x 1 root root 4.0K May  9 13:35 sample_data
-rw-r--r-- 1 root root 1.4G May 10 21:44 wiki2.en.50.bin
-rw-r--r-- 1 root root 1.4G May 10 21:47 wiki2.en.50.txt
-rw-r--r-- 1 root root 1.4G May 10 20:58 wiki.en.50.bin
-rw-r--r-- 1 root root  196 May 10 19:10 zora-20000.fasttext.tsv
-rw-r--r-- 1 root root  196 May 10 18:40 zora_20000.jsonl
-rw-r--r-- 1 root root  17K May 10 22:43 zora-eng-dewey-10.lemmatized.fasttext.tsv
-rw-r--r-- 1 root root  16M May 10 19:11 zora-eng-dewey.fasttext.tsv
-rw-r--r-- 1 root root 1.5M May 10 22:48 zora-eng-dewey.lemmatized.fasttext.test.tsv
-rw-r--r-- 1 root root  14M May 10 22:48 zora-eng-dewey.lemmatized.fasttext.train.tsv
-rw-r--r-- 1 root root  16M May 10 22:48 zora-eng-dewey.lemmatized.fasttext.tsv


In [ ]:
test_data = []
with open("zora-eng-dewey.lemmatized.fasttext.test.tsv", mode="r",encoding="utf-8") as testfile:
    for line in testfile:
        test_data.append(line.strip().split("\t"))
test_data[:3]


[['__label__570 __label__610',
  'perspective : chain dynamic of unfold and intrinsically disorder protein from nanosecond fluorescence correlation spectroscopy combine with single - molecule fret the dynamic of unfolded protein be important both for the process of protein fold and for the behavior of intrinsically disorder protein . however , method for investigate the global chain dynamic of these structurally diverse system have be limit . a versatile experimental approach be single - molecule spectroscopy in combination with förster resonance energy transfer and nanosecond fluorescence correlation spectroscopy . the concept of polymer physics offer a powerful framework both for interpret the result and for understanding and classify the property of unfold and intrinsically disorder protein . this information on long - range chain dynamic can be complement with spectroscopic technique that probe different length scale and time scale , and integration of these result greatly benefit 

In [ ]:
from collections import Counter
confusion_matrix = Counter()

# If given a list of strings, it will return a list of results as usually received for a single line of text.
predictions,probs = model.predict([text for _,text in test_data],k=3,threshold=0.25)

for i,preds in enumerate(predictions):
    labels = " ".join(sorted(preds))
    if not labels:
        print(test_data[i][0],test_data[i][1])
    confusion_matrix[(test_data[i][0],labels)] += 1

for p in confusion_matrix.most_common():
    print(p)

__label__150 item - focused trees for the detection of differential item functioning in partial credit models various method to detect differential item function ( dif ) in item response model be available . however , most of these method assume that the response be binary , and so for order response category available method be scarce . in the present article , dif in the widely use partial credit model be investigate . an item - focus tree be propose that allow the detection of dif item , which might affect the performance of the partial credit model . the method use tree methodology , yield a tree for each item that be detect as dif item . the visualization as tree make the result easily accessible , as the obtain tree show which variable induce dif and in which way . in the present paper , the new method be compare with alternative approach and simulation demonstrate the performance of the method .
__label__820 alien terrorist : public discourse on 9/11 and the american science fic

# Verbessern des Modells
Verbessern des Modells: Z.B. mehr Epochen, mehr Dimensionen, längere Buchstaben-N-Gramme, ...

Wichtigste Parameter:
```
   epoch N  # Beim Lernen wird das ganze Trainingsset N mal benutzt. Beeinflusst die Dauer des Trainings linear!
   dim N    # Länge der gelernten Vektoren für Wörter und Buchstaben-N-Gramme
   lr 0.N   # Initiale Lernrate: Bestimmt, wie stark die Vektoren verändert werden, wenn Fehler passieren. Während des Lernens wird die Lernrate immer kleiner.
   mmin N   # Minimale Länge der Subwords, d.h. Buchstaben-N-Gramme
   maxn N   # Maximale Länger der Subwords, d.h. Buchstaben-N-Gramme (falls N=0, werden keine Subwords benutzt, nur Wörter)
```

In [ ]:
model = fasttext.train_supervised(
    input='impresso-lid-train.tsv', 
    epoch=10,  # Wie oft werden die Trainingsdaten benutzt
    minn=2,    # Minimale Subword-Länge in Buchstaben  
    maxn=2,    # Maximale Subword-Länge in Buchstaben 
    dim=5,     # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords
    lr=0.1     # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    )
print_results(*model.test("impresso-lid-test.tsv"))
model.test_label('impresso-lid-test.tsv',k=1)

In [ ]:
model=ft
# Store only the 100,000 most frequent words
max_words = 400000
words = model.words[:max_words]
vectors = [model[word] for word in words]

# Save the subset of words and vectors to a text file
with open("model_subset.txt", "w", encoding="utf-8") as f:
    # Write the header with the vocabulary size and vector dimensionality
    f.write(f"{max_words} {model.get_dimension()}\n")

    # Write the vectors for each word
    for word, vector in zip(words, vectors):
        vector_str = " ".join([f"{x:.6f}" for x in vector])
        f.write(f"{word} {vector_str}\n")
